## ETL Pipeline

### Extracting: Web Scraping

In [57]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os
import urllib.request
import numpy as np

import PyPDF2
import fitz # PyMuPDF library

In [2]:
url = "https://www.sga-ag.de/ergebnisse/katalogarchiv.html"

In [3]:
# function to get the BS object of an html website.
def get_html(url):
    result = requests.get(url)
    return BeautifulSoup(result.text, "html.parser")

#### Getting the catalogue Links

In [6]:
# first we have to define a function to get all the links for the different catalogues and results

def get_catalogues_results(html):
    boxes_raw = html.find_all(class_="tx-reference-teaser normal-layout")
    catalogue_boxes = list(map(lambda x: x.find_all(class_="details"), boxes_raw))
    
    # initialize an empty dataframe
    df = pd.DataFrame(columns=["type", "dates", "catalogue_link", "result_link"])
    
    # loop through each of the boxes containing the link to the catalogue/results
    for i in catalogue_boxes:
        box = i[0]
        type = box.findNext("p")
        dates = box.findNext("p")
        catalogue_link = box.findNext("a")
        result_link = catalogue_link.findNext("a")
        
        # append the content of the a and p tags to a dataframe.
        df = df.append({"type": type.text, "dates":dates.text, "catalogue_link":catalogue_link["href"], "result_link":result_link["href"]}, ignore_index=True)

    return df

In [7]:
results_catalogue_links = get_catalogues_results(get_html(url))

In [8]:
# creating the "id" column that contains the year (e.g. 21 for 2021) and the auction number (01 for the first of the 4 total auctions in that year)
patt = r"\d{2,4}[\-\_]\d+"
results_catalogue_links["id"] = results_catalogue_links["result_link"].apply(lambda x: re.findall(patt, x))
results_catalogue_links["id"] = results_catalogue_links["id"].apply(lambda x: "".join(x))

# manually change one value for which there is no id in the link
results_catalogue_links.loc[results_catalogue_links["id"] == "", "id"] = "2012-3"

# save results to csv
results_catalogue_links.to_csv("Data\\links.csv")
results_catalogue_links.tail(3)

,type,dates,catalogue_link,result_link,id
37,Herbst-Auktionen,Herbst-Auktionen,/fileadmin/user_upload/sga-ag.de/ergebnisse/ka...,/fileadmin/user_upload/sga-ag.de/ergebnisse/ka...,2012-3
38,Sommer-Auktionen,Sommer-Auktionen,/fileadmin/user_upload/sga-ag.de/ergebnisse/ka...,/fileadmin/user_upload/sga-ag.de/ergebnisse/ka...,2012-2
39,Winter-Auktionen,Winter-Auktionen,/fileadmin/user_upload/sga-ag.de/ergebnisse/ka...,/fileadmin/user_upload/sga-ag.de/ergebnisse/ka...,2012-4


#### Downloading PDFs with given Link and creating folders

In [9]:
# function to download the PDFs.

def download_pdf(url, filename, document_type=None):
    pdf = urllib.request.urlopen(url)
    
    if document_type == None:
        raise Exception("Is the pdf a result or a catalogue?")
    elif document_type == "result":
        path = "Data\\results\\"
        os.mkdir("Data\\results") if "results" not in os.listdir("Data") else None
    elif document_type == "catalogue":
        path = "Data\\catalogue\\"
        os.mkdir("Data\\catalogue") if "catalogue" not in os.listdir("Data") else None
    
    with open(path + filename, "wb") as f:
        f.write(pdf.read())

#### Catalogue PDFs

In [ ]:
# go through the entire list of links
for i in range(len(results_catalogue_links)):
    
    for j in ["catalogue_link", "result_link"]:
        
        # get the link to the website where the PDF is stored
        link = "https://www.sga-ag.de" + results_catalogue_links.iloc[i][j]
        kind = j.split("_")[0]
        
        # create the file name
        file_name = kind + "_" + results_catalogue_links.iloc[i]["id"] + ".pdf"
        
        # download the pdf and write it to the correct path
        download_pdf(link, file_name, document_type=j.split("_")[0])
        
        # print out the progress
        print("download finished: ", link, file_name)
else:
    print("Finished")

### Transforming: getting information about the auction objects from the raw PDFs

Extracting all the information from the PDFs is quite a tedious task.

Problems:
    - different PDF structure that changed over the years.

Some regularities concerning the structure of the docuemnts:

- the description for a single auction object always ends with the "Mindestgebot"
- every auction object starts with "Lage:"
- besides that the structure of the listings can vary depending on the type of real estate, attributes can be missing.

Approach:

- split the contents of a page by "Lage", since every auction item has this at the beginning of the decription it will seperate the different auction items even if there are multiple ones on one page.

- instead of searching for the properties of the objects directly we will split by the attribute resulting in a list with an even number of items

- general structure of an offering:
    - Location -> Lage >> Grundstücksgröße, Objektbeschreibung
    - (Estate Size)
    - Object Description >> Energieausweis, Mindestgebot, Fläche
    - (living/floor space) "Wohn-/Nutzfläche" or "Wohn-/ Nutzfläche" or "Wohnfläche" or "Nutzfläche"
    - (Energy Certificate) -> sometimes not available for older offerings 
    - (Yearly Rent (net or gross))
    - (housing benefits) -> "Wohngeld"/"Hausgeld mtl."
    - Minimal bid

In [68]:
def finder(pattern, pdf):

    res = re.findall(pattern, pdf)
    print(res)
    if res == []:
        return [np.NaN]
    else:
        return res
    
def get_pdf_contents(pdf: str, df: pd.DataFrame):

    # firstly, seperate the page into the different auction object that might be on it
    
    # every object begins with "Lage", the first value could potentially contain the ID of the auction object
    res = re.split(r"(Lage:)", pdf)
    id_info = res[0]
    objects = res[1:]
    
    # put the objects into a list [description]]
    objects = [objects[i*2+1] for i in list(range(0, int(len(objects)/2)))]
    
    # create the pattern for the split operation
    Attributes = ["Bruttogeschossfläche:", "Grundstücksgröße:", "Objektbeschreibung:", "Lage:", "Energieausweis:" ,"Nutzfläche:", "Hausgeld mtl.:", "Mindestgebot:", "Wohnfläche:"]
    
    # some of the search tags have special regex characters like the "()" in "Jahresmiete (netto)" we have to therefore manually ad them.
    search = "(" + "|".join(Attributes) + "|Jahresmiete \(netto\):" + ")"
    split_pattern = re.compile(search)
    
    Lagen = []
    rest_dict = []
    
    for object in objects:
        
        split_object = re.split(split_pattern, object)
        
        rest = split_object[1:]
        
        d = {rest[i*2]: rest[i*2+1] for i in range(int(len(rest)/2))}
        d["Lage:"] = split_object[0]
        rest_dict.append(d)
        
        
    df = df.append(rest_dict, ignore_index=True)
    return df


# helper function to get the string of the PDF page and remove extra spaces
def pdf_page_as_string(pdf, page: int):
        return " ".join(pdf.getPage(page).extractText().split()).replace("- ", "")

In [ ]:
# get all the relevant paths where the PDFs are stored
paths = os.listdir(path="Data\\catalogue")

# instantiate a new Pandas Dataframe in which we will store the data
data = pd.DataFrame()

for path in paths:
    
    # if reading of the PDF document fails we will skipp to the next one since that is most likely a problem with the PyPDF2 library for which there is not fix.
    try:
        pdf_object = open("Data\\catalogue\\" + path, "rb")
        pdf_reader = PyPDF2.PdfFileReader(pdf_object)
    except:
        print("Problem Handling Document Number: {}".format(path))
    
    print("Currently Processing: {}".format(path))
    num_pages = pdf_reader.getNumPages()
    
    # loop throug all of the pages of the PDF document
    for i in range(num_pages):
        pdf_string = pdf_page_as_string(pdf_reader, i)
        
        # append the data gathered from the page to the PDF document.
        data = get_pdf_contents(pdf_string, data)

#### Extracting Data from the "results" PDFs

Unfortunately, the library PyDF2 did not work with the results PDFs, therefore we will fall back on the PyMuPDF library.

The general structure of these results is:

- Object Number
- City
- Adress
- Limit Price
- Winning bid

In [50]:
doc = fitz.open("Data\\results\\" + "result_18-01.pdf")   

In [67]:
page = doc.load_page(1)

In [82]:
patt = r"(\d{1,2}\.)\n"
# for i in re.split(patt, page.get_text()):
#     print(i)
#     
    
re.split(patt, page.get_text())[1:]

['25.',
 'Eisfeld\nTraubenwirtsgasse 13\n1.000,--\n5.000,--\n',
 '26.',
 'Gräfenthal\nProbstzellaer Straße 11-13a\n125.000,--\n131.000,--\n',
 '27.',
 'Oberweißbach\nFröbelstraße/ Unter dem Kohlwege\n9.000,--\n9.010,--\n',
 '29.',
 'Bleicherode\nNordhäuser Straße 93\n9.000,--\n23.000,--\n',
 '30.',
 'Saara OT Geißen\nFlurstück 114\n90.000,--\n102.000,--\n',
 '31.',
 'Stadt Sandersleben (Anhalt)\nAscherslebener Straße 24\n25.000,--\n25.000,--\n',
 '32.',
 'Mansfeld OT Möllendorf\nMöllendorfer Dorfstraße 28\n9.000,--\n20.500,--\n',
 '33.',
 'Köthen\nGeorgstraße 2a-c\n9.000,--\n105.000,--\n',
 '34.',
 'Köthen OT Baasdorf\nLindenplatz 8\n12.000,--\n20.000,--\n',
 '35.',
 'Muldestausee OT Pouch\nFeldstraße 1\n49.000,--\n80.000,--\n',
 '36.',
 'Burg\nMarkt 20\n25.000,--\n39.000,--\n',
 '37.',
 'Oschersleben OT Schmercke\nAm Jungfernhölzchen 7, 8, 9, 10 und 11\n15.000,--\n15.000,--\n',
 '38.',
 'Kriebitzsch\nKurze Straße\n1.000,--\n2.000,--\n',
 '39.',
 'Hecklingen OT Schneidlingen\nAm Bahnho

### Loading Save results to a MongoDB database

We want to have a flexible schema that depends on how many attributes we find for a given auction object.

In [54]:
toc

[]

In [11]:
pdf_object = open("Data\\results\\" + "result_18-01.pdf", "rb")
pdf_reader = PyPDF2.PdfFileReader(pdf_object)

In [21]:
page = pdf_reader.getPage(4)

In [46]:
page.mediaBox

RectangleObject([0, 0, 595, 842])